In [1]:
!pip install fastai
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 63kB/s  eta 0:00:01MB 17.3MB/s eta 0:00:31    29% |█████████▌                      | 171.1MB 36.7MB/s eta 0:00:12    42% |█████████████▋                  | 244.9MB 43.0MB/s eta 0:00:08████                  | 251.4MB 18.0MB/s eta 0:00:19    44% |██████████████▎                 | 256.8MB 40.8MB/s eta 0:00:08    45% |██████████████▋                 | 262.6MB 33.1MB/s eta 0:00:10    50% |████████████████▎               | 292.4MB 40.7MB/s eta 0:00:07    55% |█████████████████▋              | 317.8MB 44.4MB/s eta 0:00:0623.2MB 40.0MB/s eta 0:00:04�█████████████████▌| 567.5MB 38.9MB/s eta 0:00:01��████████████▋| 569.3MB 5.8MB/s eta 0:00:02�█████████████▋| 569.6MB 6.3MB/s eta 0:00:02
You are using pip version 19.0.3, ho

In [2]:
import numpy as np 
import pandas as pd 
from fastai import *
from fastai.text import *

In [3]:
gita = pd.read_csv('../input/bhagavad-gita.csv')

In [4]:
print(gita.head)
print(gita.keys())

<bound method NDFrame.head of      Unnamed: 0                        ...                                                      verse_text_no_samdhis
0             0                        ...                          dhṛtarāṣṭraḥ uvāca dharma kṣetre kuru kṣetre s...
1             1                        ...                          sañjayaḥ uvāca dṛṣṭvā tu pāṇḍava anīkam vyūḍha...
2             2                        ...                          paśya etām pāṇḍu putrānām ācārya mahatīm camūm...
3             3                        ...                          atra śūrāḥ mahā iṣu āsāḥ bhīma arjuna samāḥ yu...
4             4                        ...                          dhṛṣṭaketuḥ cekitānaḥ kāśirājaḥ ca vīrya vān p...
5             5                        ...                          yudhāmanyuḥ ca vikrāntaḥ uttamaujāḥ ca vīrya v...
6             6                        ...                          asmākam tu viśiṣṭāḥ ye tān nibodha dvija uttam...
7             7           

In [5]:
print(gita.shape)

(700, 5)


In [ ]:
#quote = quote.drop(quote.columns[[0,3,4,5]], axis=1)

In [24]:
gita['devanagari'] = gita['devanagari'].str.lower()
print(gita.head())

   Unnamed: 0                        ...                                                      verse_text_no_samdhis
0           0                        ...                          dhṛtarāṣṭraḥ uvāca dharma kṣetre kuru kṣetre s...
1           1                        ...                          sañjayaḥ uvāca dṛṣṭvā tu pāṇḍava anīkam vyūḍha...
2           2                        ...                          paśya etām pāṇḍu putrānām ācārya mahatīm camūm...
3           3                        ...                          atra śūrāḥ mahā iṣu āsāḥ bhīma arjuna samāḥ yu...
4           4                        ...                          dhṛṣṭaketuḥ cekitānaḥ kāśirājaḥ ca vīrya vān p...

[5 rows x 5 columns]


In [25]:
from pandas import DataFrame
verse = DataFrame.drop_duplicates(DataFrame(gita, columns= ['devanagari']))
verse.head()

,devanagari
0,धृतराष्ट्र उवाच ।\r\nधर्मक्षेत्रे कुरुक्षेत्रे...
1,सञ्जय उवाच ।\r\nदृष्ट्वा तु पाण्डवानीकं व्यूढं...
2,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् ।\r...
3,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि ।\r\nयुय...
4,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान् ।\r\n...


In [26]:
test_per = 0.05        # test percentage
verse = verse.iloc[np.random.permutation(len(verse))]
piece = int(test_per * len(verse)) + 1
train_verse, test_verse = verse[piece:], verse[:piece]

In [27]:
print(len(train_verse),len(test_verse))

664 36


In [29]:
# Pre-processing
nan_row = verse[verse['devanagari'].isnull()]
print(nan_row)

Empty DataFrame
Columns: [devanagari]
Index: []


In [31]:
data_lm = TextLMDataBunch.from_df('content', train_verse, test_verse, text_cols = 'devanagari')
#data_clas = TextClasDataBunch.from_df('content', train_quote, test_quote, vocab = data_lm.train_ds.vocab, bs = 32)
#data_lm.save('data_lm_export.pkl')
#data_clas.save('data_clas_export.pkl')

In [53]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult = 0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,8.315605,6.984680,0.333929,00:00


In [54]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.918251,4.567159,0.344196,00:00


In [55]:
learn.fit(20, lr=1e-4, wd=1e-7)

epoch,train_loss,valid_loss,accuracy,time
0,5.127543,4.205128,0.437723,00:00
1,4.956701,4.233127,0.440179,00:00
2,4.883216,4.348653,0.442634,00:00
3,4.823263,4.262582,0.442411,00:00
4,4.752398,4.116508,0.443304,00:00
5,4.686560,3.966731,0.451562,00:00
6,4.646135,3.898230,0.436161,00:00
7,4.597675,3.885762,0.420982,00:00
8,4.557968,3.859381,0.416741,00:00
9,4.519495,3.796874,0.447098,00:00


In [60]:
number_of_quotes = 50
quotes_counter = 0
all_quotes = []

for i in range(1000):
    quote = learn.predict("xxbos ", n_words=20, temperature=0.8)
    quotes = quote.split("xxbos ")
    quotes = quotes[1:-1]
    
    for quote in quotes:
        if ("\r" or "\n" or "॥") in quote:
            quote = quote.replace("\r","")
            quote = quote.replace("\n", "")
            quote = quote.replace("॥","")
        quote = quote.replace("xxbos ","").strip()
        quote = quote.replace("Xxbos ","").strip()
        if(quote):
            all_quotes.append(quote)
            quotes_counter = quotes_counter+1
            
    if quotes_counter > number_of_quotes:
        break

In [61]:
all_quotes

['तत्त्वतः',
 'स्थानं दीयते जगत्कृत्स्नं ज्ञात्वा ज्ञानेन युगे हृषीकेशं रणे संयाति यं परन्तप सर्वस्य',
 'एवायं परं विष्णो संन्यासं सुखम् समुपस्थितम् मम चेति सर्वाणि पुरुषोत्तम शृणु दीयते सत्त्वं',
 'संजय',
 'Xxbos',
 'Xxbos',
 'मन्ये विष्णो कर्मणां नित्ययुक्ता मय्येव यया वर्त्मानुवर्तन्ते योगः',
 'देवा स्थानं पुनः कारणमुच्यते दानं मनसा सः परा ॥ निबध्नन्ति पतन्ति ज्ञेयं ददामि निगच्छति भवार्जुन विचाल्यते',
 'नैव देही अधिभूतं निधानम् हृदि मनुष्याः कौन्तेय सर्वतः जुह्वति तमसः सर्वे पार्थ प्रपद्यन्ते यया जायन्ते कुर्वन्ति',
 'दिवि किम् न हन्यते युक्त मम उच्यते सर्वभूतानां विवृद्धे सर्वारम्भपरित्यागी ॥',
 'सञ्जय कुलघ्नानां जन्म निर्ममो तु पुनर्जन्म बुद्धेः गच्छति मनो कर्म एवमुक्त्वा शीतोष्णसुखदुःखेषु सात्त्विका राजसाः संजय स्थितं',
 'निबध्नन्ति लभन्ते तानि दिवि विद्धि वक्त्राणि धनंजय मुच्यन्ते']

In [69]:
outF = open("verses.txt", "w")
j = 0
for verse in all_quotes:
  # write line to output file
  j=j+1
  outF.write(verse)
  outF.write("\n "+str(j)+" \n\n")
outF.close()